<a href="https://colab.research.google.com/github/xuanyu410/114-1-PLproject/blob/main/%E4%BD%A0%E8%A7%80%E6%98%9F%E5%97%8E%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ========== 📌 CELL 1：安裝套件 ==========
!pip install -q streamlit streamlit-folium folium wordcloud beautifulsoup4 requests pandas matplotlib pyngrok jieba google-generativeai gspread Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.5 MB/s eta 0:00:00


In [2]:
# ============================================
# 方法 1：完全停止並重新啟動（推薦）
# ============================================

# STEP 1：停止所有正在運行的 Streamlit 和 ngrok
import subprocess
import os
import signal
from pyngrok import ngrok

print("🛑 正在停止舊的服務...")

# 關閉 ngrok
ngrok.kill()

# 殺掉所有 streamlit 進程
try:
    subprocess.run(["pkill", "-9", "-f", "streamlit"], check=False)
    print("✅ 已停止所有 Streamlit 進程")
except:
    pass

import time
time.sleep(3)

print("✅ 清理完成！")

🛑 正在停止舊的服務...
✅ 已停止所有 Streamlit 進程
✅ 清理完成！


In [3]:
# ========== 📌 CELL 2：測試套件 ==========
import streamlit
import folium
from bs4 import BeautifulSoup
print("✅ 所有套件安裝成功！")



✅ 所有套件安裝成功！


In [4]:
# ========== 📌 CELL 3：創建主程式 app.py ==========
%%writefile app.py
import streamlit as st
import requests
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import folium
from streamlit_folium import st_folium
from collections import Counter
import re
import jieba
import time

st.set_page_config(
    page_title="你觀星嗎？",
    page_icon="🌌",
    layout="wide"
)

st.markdown("""
    <style>
    .main {
        background-color: #0a0e27;
        color: #f5f5dc;
    }
    .stApp {
        background: linear-gradient(180deg, #0a0e27 0%, #1a1f3a 100%);
    }
    h1, h2, h3 {
        color: #64b5f6 !important;
    }
    p, .stMarkdown, .stText, label {
        color: #f5f5dc !important;
    }
    .stButton>button {
        background-color: #1976d2;
        color: white;
        border-radius: 10px;
        padding: 0.5rem 2rem;
        font-size: 1.1rem;
        transition: all 0.3s ease;
    }
    .stButton>button:hover {
        background-color: #1565c0;
        transform: scale(1.05);
        box-shadow: 0 0 20px rgba(100, 181, 246, 0.5);
    }
    /* Sidebar 樣式 */
    [data-testid="stSidebar"] {
        background: linear-gradient(180deg, #1a1f3a 0%, #2a2f4a 100%);
    }
    [data-testid="stSidebar"] .stMarkdown {
        color: #e0e0e0 !important;
    }
    [data-testid="stSidebar"] h1 {
        color: #64b5f6 !important;
    }
    [data-testid="stSidebar"] .stRadio > label {
        color: #e0e0e0 !important;
    }
    [data-testid="stSidebar"] [role="radiogroup"] label {
        color: #e0e0e0 !important;
    }

    /* 首頁動畫樣式 */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(30px); }
        to { opacity: 1; transform: translateY(0); }
    }
    @keyframes twinkle {
        0%, 100% { opacity: 0.3; }
        50% { opacity: 1; }
    }
    @keyframes float {
        0%, 100% { transform: translateY(0px); }
        50% { transform: translateY(-20px); }
    }
    .home-title {
        font-size: 4rem;
        font-weight: bold;
        text-align: center;
        background: linear-gradient(45deg, #64b5f6, #90caf9, #bbdefb);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        animation: fadeIn 2s ease-out, float 3s ease-in-out infinite;
        margin: 2rem 0;
    }
    .home-subtitle {
        font-size: 1.5rem;
        text-align: center;
        color: #90caf9;
        animation: fadeIn 2.5s ease-out;
        margin-bottom: 3rem;
    }
    .feature-card {
        background: linear-gradient(135deg, #1a237e 0%, #283593 100%);
        border-radius: 15px;
        padding: 2rem;
        margin: 1rem 0;
        border: 2px solid #64b5f6;
        animation: fadeIn 3s ease-out;
        transition: all 0.3s ease;
        cursor: pointer;
    }
    .feature-card:hover {
        transform: translateY(-10px);
        box-shadow: 0 10px 30px rgba(100, 181, 246, 0.4);
        border-color: #90caf9;
    }
    .feature-icon {
        font-size: 3rem;
        text-align: center;
        margin-bottom: 1rem;
        animation: twinkle 2s ease-in-out infinite;
    }
    .feature-title {
        font-size: 1.8rem;
        color: #64b5f6;
        text-align: center;
        font-weight: bold;
        margin-bottom: 0.5rem;
    }
    .feature-desc {
        color: #e0e0e0;
        text-align: center;
        font-size: 1.1rem;
    }
    .stars {
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        pointer-events: none;
        z-index: -1;
    }
    .star {
        position: absolute;
        background: white;
        border-radius: 50%;
        animation: twinkle 3s ease-in-out infinite;
    }
    </style>
    """, unsafe_allow_html=True)

# 初始化 session_state
if "current_page" not in st.session_state:
    st.session_state.current_page = "home"

# 首頁
if st.session_state.current_page == "home":
    # 添加星星背景效果
    st.markdown("""
        <div class="stars">
            <div class="star" style="top: 10%; left: 20%; width: 2px; height: 2px; animation-delay: 0s;"></div>
            <div class="star" style="top: 20%; left: 80%; width: 3px; height: 3px; animation-delay: 0.5s;"></div>
            <div class="star" style="top: 30%; left: 50%; width: 2px; height: 2px; animation-delay: 1s;"></div>
            <div class="star" style="top: 50%; left: 10%; width: 3px; height: 3px; animation-delay: 1.5s;"></div>
            <div class="star" style="top: 60%; left: 70%; width: 2px; height: 2px; animation-delay: 2s;"></div>
            <div class="star" style="top: 80%; left: 30%; width: 3px; height: 3px; animation-delay: 2.5s;"></div>
            <div class="star" style="top: 15%; left: 60%; width: 2px; height: 2px; animation-delay: 1.2s;"></div>
            <div class="star" style="top: 70%; left: 85%; width: 2px; height: 2px; animation-delay: 0.8s;"></div>
        </div>
    """, unsafe_allow_html=True)

    st.markdown('<div class="home-title"> 你觀星嗎？</div>', unsafe_allow_html=True)
    st.markdown('<div class="home-subtitle">✨ 探索浩瀚星空，開啟天文之旅 ✨</div>', unsafe_allow_html=True)

    # 功能選單
    col1, col2 = st.columns(2)

    with col1:
        st.markdown("""
            <div class="feature-card">
                <div class="feature-icon">🌟</div>
                <div class="feature-title">天頂星座查詢</div>
                <div class="feature-desc">輸入座標與時間，查詢天頂星座資訊</div>
            </div>
        """, unsafe_allow_html=True)
        if st.button("進入天頂星座", key="btn1", use_container_width=True):
            st.session_state.current_page = "天頂星座"
            st.rerun()

        st.markdown("""
            <div class="feature-card">
                <div class="feature-icon">📊</div>
                <div class="feature-title">熱門話題分析</div>
                <div class="feature-desc">爬取天文網站，分析熱門關鍵字</div>
            </div>
        """, unsafe_allow_html=True)
        if st.button("進入熱門話題", key="btn3", use_container_width=True):
            st.session_state.current_page = "熱門話題"
            st.rerun()

    with col2:
        st.markdown("""
            <div class="feature-card">
                <div class="feature-icon">💬</div>
                <div class="feature-title">AI 天文問答</div>
                <div class="feature-desc">使用 Gemini AI 回答天文問題</div>
            </div>
        """, unsafe_allow_html=True)
        if st.button("進入 AI 問答", key="btn2", use_container_width=True):
            st.session_state.current_page = "AI問答"
            st.rerun()

        st.markdown("""
            <div class="feature-card">
                <div class="feature-icon">📅</div>
                <div class="feature-title">觀星日誌</div>
                <div class="feature-desc">記錄每一次觀星體驗</div>
            </div>
        """, unsafe_allow_html=True)
        if st.button("進入觀星日誌", key="btn4", use_container_width=True):
            st.session_state.current_page = "觀星日誌"
            st.rerun()

    st.markdown("<br><br>", unsafe_allow_html=True)
    st.markdown('<div style="text-align: center; color: #64b5f6; font-size: 1.2rem;">🔭 讓我們一起仰望星空 </div>', unsafe_allow_html=True)

# 其他頁面
else:
    with st.sidebar:
        st.title("🔭 星空導覽")
        st.markdown("---")

        if st.button("🏠 返回首頁", use_container_width=True):
            st.session_state.current_page = "home"
            st.rerun()

        st.markdown("---")
        page = st.radio("選擇功能", ["🌟 天頂星座", "💬 AI 天文問答", "📊 熱門話題", "📅 觀星日誌"],
                       index=["天頂星座", "AI問答", "熱門話題", "觀星日誌"].index(st.session_state.current_page)
                       if st.session_state.current_page in ["天頂星座", "AI問答", "熱門話題", "觀星日誌"] else 0)

        # 更新當前頁面
        page_map = {
            "🌟 天頂星座": "天頂星座",
            "💬 AI 天文問答": "AI問答",
            "📊 熱門話題": "熱門話題",
            "📅 觀星日誌": "觀星日誌"
        }
        st.session_state.current_page = page_map[page]

        st.markdown("---")
        st.info("💡 選擇左側功能開始探索星空！")

    if st.session_state.current_page == "天頂星座":
        st.title("🌟 天頂星座查詢")
        st.markdown("### 查看所在位置當前的天頂星座")
        col1, col2 = st.columns([2, 1])
        with col1:
            st.subheader("📍 選擇觀測地點")
            user_lat = st.number_input("緯度", value=25.0330, format="%.4f")
            user_lon = st.number_input("經度", value=121.5654, format="%.4f")
            m = folium.Map(location=[user_lat, user_lon], zoom_start=10)
            folium.Marker([user_lat, user_lon], popup="觀測位置", icon=folium.Icon(color='blue', icon='star')).add_to(m)
            st_folium(m, width=700, height=400)
        with col2:
            st.subheader("🕐 觀測時間")
            obs_date = st.date_input("日期", datetime.now())
            obs_time = st.time_input("時間", datetime.now().time())
            if st.button("🔍 查詢天頂星座", type="primary"):
                st.success("查詢中...")
                st.markdown("### 🌌 天頂星座：**天琴座 (Lyra)**")
                st.markdown("**天琴座**是北天星座之一\n\n- ⭐ 主星：織女星（Vega）\n- 🌟 亮度：0.03 等\n- 📅 最佳觀測月份：7-9月")

    elif st.session_state.current_page == "AI問答":
        st.title("💬 AI 天文知識問答")
        st.markdown("### 使用 Gemini AI 回答天文問題")

        if "gemini_api_key" not in st.session_state:
            st.session_state.gemini_api_key = ""

        gemini_api_key = st.text_input(
            "🔑 輸入您的 Gemini API Key",
            value=st.session_state.gemini_api_key,
            type="password",
            key="api_key_input"
        )

        if gemini_api_key:
            st.session_state.gemini_api_key = gemini_api_key

        if "chat_history" not in st.session_state:
            st.session_state.chat_history = []

        for message in st.session_state.chat_history:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

        user_question = st.chat_input("問我任何天文問題...")

        if user_question:
            with st.chat_message("user"):
                st.markdown(user_question)
            st.session_state.chat_history.append({"role": "user", "content": user_question})

            with st.chat_message("assistant"):
                if st.session_state.gemini_api_key:
                    try:
                        import google.generativeai as genai

                        genai.configure(api_key=st.session_state.gemini_api_key)
                        model = genai.GenerativeModel('gemini-pro')

                        prompt = f"請以天文專家的角度回答以下問題：{user_question}"
                        response = model.generate_content(prompt)

                        answer = response.text
                        st.markdown(answer)
                        st.session_state.chat_history.append({"role": "assistant", "content": answer})
                    except Exception as e:
                        error_msg = f"❌ API 呼叫失敗：{str(e)}\n\n請確認 API Key 是否正確。"
                        st.error(error_msg)
                        st.session_state.chat_history.append({"role": "assistant", "content": error_msg})
                else:
                    st.warning("⚠️ 請先輸入 Gemini API Key")

    elif st.session_state.current_page == "熱門話題":
        st.title("📊 近期天文熱門話題")

        if st.button("🔄 抓取最新話題", type="primary"):
            with st.spinner("正在爬取資料..."):
                try:
                    url = "https://www.astronomy.com"
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
                    response = requests.get(url, headers=headers, timeout=10)
                    soup = BeautifulSoup(response.content, 'html.parser')

                    titles = []
                    for tag in soup.find_all(['h1', 'h2', 'h3', 'a']):
                        text = tag.get_text(strip=True)
                        if 10 < len(text) < 200:
                            titles.append(text)

                    stopwords_en = ['the', 'and', 'for', 'with', 'this', 'that', 'from', 'are', 'was', 'were', 'have', 'has']
                    stopwords_zh = ['的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都', '一', '一個', '上', '也', '很', '到', '說', '要', '去', '你', '會', '著', '沒有', '看', '好', '自己', '這']

                    all_text = ' '.join(titles)

                    words_en = [w.lower() for w in re.findall(r'\b[a-zA-Z]+\b', all_text)
                               if w.lower() not in stopwords_en and len(w) > 3]

                    chinese_text = re.sub(r'[a-zA-Z0-9\s]+', '', all_text)
                    words_zh = [w for w in jieba.cut(chinese_text)
                               if w not in stopwords_zh and len(w) > 1 and w.strip()]

                    all_words = words_en + words_zh
                    word_freq = Counter(all_words)

                    col1, col2 = st.columns([2, 1])

                    with col1:
                        st.subheader("☁️ 熱門關鍵字文字雲")
                        if word_freq:
                            wordcloud = WordCloud(
                                width=800,
                                height=400,
                                background_color='#0a0e27',
                                colormap='Blues',
                                font_path=None
                            ).generate_from_frequencies(word_freq)

                            fig, ax = plt.subplots(figsize=(10, 5))
                            ax.imshow(wordcloud, interpolation='bilinear')
                            ax.axis('off')
                            st.pyplot(fig)
                        else:
                            st.warning("未找到足夠的關鍵字")

                    with col2:
                        st.subheader("🔥 Top 10 熱門關鍵字")
                        if word_freq:
                            df = pd.DataFrame(word_freq.most_common(10), columns=['關鍵字', '次數'])
                            st.dataframe(df, use_container_width=True)
                        else:
                            st.info("暫無數據")

                    st.subheader("📰 最新文章")
                    for i, title in enumerate(titles[:10], 1):
                        st.markdown(f"{i}. {title}")

                    st.success("✅ 資料抓取完成！")

                except Exception as e:
                    st.error(f"❌ 爬取失敗：{str(e)}")

    elif st.session_state.current_page == "觀星日誌":
        st.title("📅 我的觀星日誌")

        if 'logs' not in st.session_state:
            st.session_state.logs = []

        with st.expander("➕ 新增觀星記錄", expanded=True):
            col1, col2 = st.columns(2)
            with col1:
                log_date = st.date_input("📅 觀測日期", datetime.now())
                log_location = st.text_input("📍 觀測地點", "台北")
                log_weather = st.select_slider("🌤️ 天氣", options=["多雲", "晴朗", "極佳"])
            with col2:
                log_constellation = st.text_input("🌟 觀測星座", "獵戶座")
                log_equipment = st.text_input("🔭 器材", "雙筒望遠鏡")
                log_rating = st.slider("⭐ 評分", 1, 5, 4)

            log_notes = st.text_area("📝 筆記", placeholder="記錄觀測心得...")

            if st.button("💾 儲存記錄", type="primary"):
                new_log = {
                    "date": log_date.strftime("%Y-%m-%d"),
                    "location": log_location,
                    "weather": log_weather,
                    "constellation": log_constellation,
                    "equipment": log_equipment,
                    "rating": log_rating,
                    "notes": log_notes
                }
                st.session_state.logs.append(new_log)
                st.success("✅ 記錄已儲存！")

        if st.session_state.logs:
            st.subheader("📖 觀星記錄")
            for log in reversed(st.session_state.logs):
                with st.container():
                    col1, col2, col3 = st.columns([2, 2, 1])
                    with col1:
                        st.markdown(f"**📅 {log['date']}** | 📍 {log['location']}")
                    with col2:
                        st.markdown(f"🌟 {log['constellation']} | ⭐ {'★' * log['rating']}")
                    with col3:
                        st.markdown(f"🌤️ {log['weather']}")
                    if log['notes']:
                        st.markdown(f"💬 {log['notes']}")
                    st.markdown("---")

            st.subheader("📊 統計")
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("總次數", len(st.session_state.logs))
            with col2:
                avg = sum(l['rating'] for l in st.session_state.logs) / len(st.session_state.logs)
                st.metric("平均評分", f"{avg:.1f}⭐")
            with col3:
                st.metric("星座數", len(set(l['constellation'] for l in st.session_state.logs)))
        else:
            st.info("📭 開始記錄觀星之旅吧！")

st.markdown("---")
st.markdown('<div style="text-align: center; color: #64b5f6;"><p>🌌 你觀星天文嗎？| Made with Streamlit ✨</p></div>', unsafe_allow_html=True)

Writing app.py


In [6]:
# ========== 📌 CELL 4：啟動 Streamlit（使用 ngrok）==========
from pyngrok import ngrok
import subprocess
import time

# 🔑 替換成你的 ngrok authtoken
# 到 https://dashboard.ngrok.com/get-started/your-authtoken 取得
NGROK_TOKEN = "35dOLyEekJ87tjKx2BGiUjl1Loo_6qFqp1TvLwXugTeb8bWRk"  # 👈 在這裡貼上你的 token

print("🚀 正在啟動...")

# 設定 token
ngrok.set_auth_token(NGROK_TOKEN)

# 關閉舊連線
ngrok.kill()

# 啟動 Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# 等待啟動
print("⏳ 等待 15 秒...")
time.sleep(15)

# 建立公開網址
try:
    public_url = ngrok.connect(8501)
    print("\n" + "="*70)
    print("🎉 成功啟動！")
    print("="*70)
    print(f"\n🔗 公開網址：{public_url}")
    print("\n💡 提示：")
    print("   - 點擊上面的網址即可訪問")
    print("   - 要停止請選擇 Runtime > Restart Runtime")
    print("\n" + "="*70)
except Exception as e:
    print(f"❌ 錯誤：{e}")
    print("請確認 ngrok token 是否正確")



🚀 正在啟動...
⏳ 等待 15 秒...


❌ 錯誤：ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://unobscure-debroah-glumpiest.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}

請確認 ngrok token 是否正確
